# Bus Tracker

Adam Beigel

24 February 2024

In [53]:
import requests
import os
import json

import pandas as pd
import numpy as np
from dotenv import load_dotenv

### Load API keys

In [3]:
load_dotenv()
bus_api_key = os.getenv('BUS_API_KEY')
train_api_key = os.getenv('TRAIN_API_KEY')

In [189]:
station = 41220

url = 'http://lapi.transitchicago.com/api/1.0/ttarrivals.aspx'
params = {
    'key': train_api_key,
    'mapid': station,
    'outputType': 'json'
}

r = requests.get(url, params=params)

In [195]:
# r.json()

In [191]:
curr_tm = r.json()['ctatt']['tmst']
curr_tm = pd.to_datetime(curr_tm)
print(curr_tm)

2024-02-25 08:40:42


In [192]:
preds = r.json()['ctatt']['eta']
df = pd.DataFrame.from_dict(preds)

df['arrT'] = pd.to_datetime(df['arrT'])
df['prdtm'] = df['arrT'].dt.strftime('%H:%M')
df['prdctdn'] = df['arrT'] - curr_tm
df['prdctdn'] = (df['prdctdn'].dt.seconds / 60).astype(np.int64).astype(str)
df.loc[df['prdctdn'].astype(int) > 1420, 'prdctdn'] = 'DUE'
df['isSch'] = df['isSch'].astype(int)

arrivals = df.sort_values(by=['rt', 'destNm', 'arrT']).loc[:, ('rt', 'destNm', 'prdtm', 'prdctdn', 'staNm', 'isSch')]
arrivals = arrivals.rename(columns={'destNm':'rtdir', 'staNm':'stpnm'})

In [193]:
def print_preds(df):
    print(f"  {df.iloc[0]['rtdir']}")
    for _, p in df.iterrows():
        min = 'min ' if p['prdctdn'] != 'DUE' else ''
        sch = 'Sch: ' if p['isSch'] == 1 else ''
        print(f"    {sch}{p['prdctdn']} {min}({p['prdtm']})")

def print_route(df):
    row = df.iloc[0]
    print(f"{row['rt']}")
    df.groupby('rtdir').apply(print_preds)
    print()

In [194]:
print(f"The time is {curr_tm.strftime('%H:%M')}\n")
arrivals.groupby('rt').apply(print_route)

The time is 08:40

Brn
  Kimball
    DUE (08:39)
    18 min (08:59)
    28 min (09:09)
    45 min (09:26)
  Loop
    7 min (08:48)
    Sch: 26 min (09:07)

Red
  95th/Dan Ryan
    8 min (08:49)
    Sch: 31 min (09:12)
  Howard
    15 min (08:55)
    36 min (09:16)



""
